In [49]:

import pandas as pd
import numpy as np
import csv
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import tensorflow as tf

from keras.layers import SimpleRNN
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import to_categorical
import keras
from keras.models import Sequential, load_model
from keras.layers.core import Dense
from keras.optimizers import RMSprop, Adam, SGD
from keras.utils import np_utils
from keras.callbacks import EarlyStopping
from tensorflow.keras import optimizers 

In [2]:
df = pd.read_csv("data/AIDA_CRM_20210312.csv", encoding = "UTF8", thousands = ",", index_col=0)
df

,아이디,나이,성별,피부타입,피부고민,선호제품,주문상품명,옵션입력,앰플
0,thsk486,4.0,0,건성,"건조함,좁쌀여드름,홍조&예민",진정앰플,[투비리본X아이다] 앰플&크림 2차 공구!,필수 앰플 쑥앰플 진정앰플 수분라이트 앰플 쑥앰플 진...,"[' 쑥앰플', ' 진정앰플', ' 쑥앰플', ' 진정앰플', ' 쑥앰플', ' 진..."
1,thsk486,4.0,0,건성,"건조함,좁쌀여드름,홍조&예민",진정앰플,[투비리본X아이다] 앰플&크림 2차 공구!,필수 앰플 쑥앰플 진정앰플 수분라이트 앰플 쑥앰플 진...,"[' 쑥앰플', ' 진정앰플', ' 쑥앰플', ' 진정앰플', ' 쑥앰플', ' 진..."
2,sjh3460,3.0,0,지성,여드름,진정앰플,[투비리본X아이다] 앰플&크림 2차 공구!,필수 앰플 쑥앰플 진정앰플 수분라이트 앰플 쑥앰플 진...,"[' 쑥앰플', ' 진정앰플', ' 쑥앰플', ' 진정앰플', ' 쑥앰플', ' 진..."
3,kaeng2da,3.0,0,복합성,좁쌀여드름,수분앰플,[투비리본X아이다] 앰플&크림 2차 공구!,필수 앰플 쑥앰플 진정앰플 수분라이트 앰플 쑥앰플 진...,"[' 쑥앰플', ' 진정앰플', ' 쑥앰플', ' 진정앰플', ' 쑥앰플', ' 진..."
4,gmlwlss625,2.0,0,복합성,"과잉피지,여드름,좁쌀여드름,홍조&예민",진정앰플,[투비리본X아이다] 앰플&크림 2차 공구!,필수 앰플 쑥앰플 진정앰플 수분라이트 앰플 쑥앰플 진...,"[' 쑥앰플', ' 진정앰플', ' 쑥앰플', ' 진정앰플', ' 쑥앰플', ' 진..."
...,...,...,...,...,...,...,...,...,...
34651,nadaseonho,3.0,0,복합성,"건조함,홍조&예민",진정앰플,[시앙제이X아이다] 인생 버블토너 1차 공구!,필수 옵션 프로폴리스 진정토너 옵션 머그워트 클리어토너 옵션 ...,[' 진정앰플울트라']
34652,rarrarra12,3.0,0,지성,"과잉피지,여드름,좁쌀여드름",진정앰플,BEST앰플3종(수분/미백/진정),필수 쿠폰할인 수분앰플 쿠폰할인 미백앰플 쿠폰할인 진정앰플,"[' 수분앰플', ' 미백앰플', ' 진정앰플']"
34653,gy6686,1.0,0,건성,"건조함,홍조&예민",진정앰플,주경민 이사님 개인결제창,본품 수분앰플오리지널 수분앰플라이트 진정앰플 모공 쑥 앰플 부스터앰플 미백앰플 진정...,"[' 수분앰플오리지널', ' 수분앰플라이트', ' 진정앰플', ' 부스터앰플', '..."
34654,gy6686,1.0,0,건성,"건조함,홍조&예민",진정앰플,주경민 이사님 개인결제창,본품 수분앰플오리지널 수분앰플라이트 진정앰플 모공 쑥 앰플 부스터앰플 미백앰플 진정...,"[' 수분앰플오리지널', ' 수분앰플라이트', ' 진정앰플', ' 부스터앰플', '..."


In [3]:
# 피부타입 확인
df["피부타입"].unique()

array(['건성', '지성', '복합성', '모름'], dtype=object)

In [4]:
# 피부타입 더미변수화 후 열 추가
encoding = pd.get_dummies(df['피부타입'])
df = pd.concat([df, encoding], axis=1)
# 원래 피부타입 제거
del df['피부타입']


In [6]:
# 좁쌀여드름 -> 좁쌀
df['피부고민'] = df['피부고민'].str.replace(pat='좁쌀여드름', repl='좁쌀', regex=False)

# 피부고민 리스트
faceworry = ['건조함','과잉피지','여드름','좁쌀','홍조&예민','기미&잡티']

# 피부고민 더미변수 열 추가
df['건조함'] = None
df['과잉피지'] = None
df['여드름'] = None
df['좁쌀'] = None
df['홍조&예민'] = None
df['기미&잡티'] = None

# 피부고민 숫자형태로 인코딩
for i in range(len(df)):
    for j in faceworry:
        if j in df['피부고민'][i]:
            df[j][i] = 1
        else:
            df[j][i] = 0
            
# 원래 피부고민 열 제거
del df['피부고민']

<ipython-input-6-a59e96ee05a4>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[j][i] = 1
<ipython-input-6-a59e96ee05a4>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[j][i] = 0


In [7]:
df

,아이디,나이,성별,선호제품,주문상품명,옵션입력,앰플,건성,모름,복합성,지성,건조함,과잉피지,여드름,좁쌀,홍조&예민,기미&잡티
0,thsk486,4.0,0,진정앰플,[투비리본X아이다] 앰플&크림 2차 공구!,필수 앰플 쑥앰플 진정앰플 수분라이트 앰플 쑥앰플 진...,"[' 쑥앰플', ' 진정앰플', ' 쑥앰플', ' 진정앰플', ' 쑥앰플', ' 진...",1,0,0,0,1,0,0,1,1,0
1,thsk486,4.0,0,진정앰플,[투비리본X아이다] 앰플&크림 2차 공구!,필수 앰플 쑥앰플 진정앰플 수분라이트 앰플 쑥앰플 진...,"[' 쑥앰플', ' 진정앰플', ' 쑥앰플', ' 진정앰플', ' 쑥앰플', ' 진...",1,0,0,0,1,0,0,1,1,0
2,sjh3460,3.0,0,진정앰플,[투비리본X아이다] 앰플&크림 2차 공구!,필수 앰플 쑥앰플 진정앰플 수분라이트 앰플 쑥앰플 진...,"[' 쑥앰플', ' 진정앰플', ' 쑥앰플', ' 진정앰플', ' 쑥앰플', ' 진...",0,0,0,1,0,0,1,0,0,0
3,kaeng2da,3.0,0,수분앰플,[투비리본X아이다] 앰플&크림 2차 공구!,필수 앰플 쑥앰플 진정앰플 수분라이트 앰플 쑥앰플 진...,"[' 쑥앰플', ' 진정앰플', ' 쑥앰플', ' 진정앰플', ' 쑥앰플', ' 진...",0,0,1,0,0,0,0,1,0,0
4,gmlwlss625,2.0,0,진정앰플,[투비리본X아이다] 앰플&크림 2차 공구!,필수 앰플 쑥앰플 진정앰플 수분라이트 앰플 쑥앰플 진...,"[' 쑥앰플', ' 진정앰플', ' 쑥앰플', ' 진정앰플', ' 쑥앰플', ' 진...",0,0,1,0,0,1,1,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34651,nadaseonho,3.0,0,진정앰플,[시앙제이X아이다] 인생 버블토너 1차 공구!,필수 옵션 프로폴리스 진정토너 옵션 머그워트 클리어토너 옵션 ...,[' 진정앰플울트라'],0,0,1,0,1,0,0,0,1,0
34652,rarrarra12,3.0,0,진정앰플,BEST앰플3종(수분/미백/진정),필수 쿠폰할인 수분앰플 쿠폰할인 미백앰플 쿠폰할인 진정앰플,"[' 수분앰플', ' 미백앰플', ' 진정앰플']",0,0,0,1,0,1,1,1,0,0
34653,gy6686,1.0,0,진정앰플,주경민 이사님 개인결제창,본품 수분앰플오리지널 수분앰플라이트 진정앰플 모공 쑥 앰플 부스터앰플 미백앰플 진정...,"[' 수분앰플오리지널', ' 수분앰플라이트', ' 진정앰플', ' 부스터앰플', '...",1,0,0,0,1,0,0,0,1,0
34654,gy6686,1.0,0,진정앰플,주경민 이사님 개인결제창,본품 수분앰플오리지널 수분앰플라이트 진정앰플 모공 쑥 앰플 부스터앰플 미백앰플 진정...,"[' 수분앰플오리지널', ' 수분앰플라이트', ' 진정앰플', ' 부스터앰플', '...",1,0,0,0,1,0,0,0,1,0


In [8]:
# 중간 저장 (CSV)
df.to_csv('data/CRM_mid.csv', encoding='utf-8-sig')

In [9]:
# 중간 불러오기
df = pd.read_csv('data/CRM_mid.csv', encoding='utf-8-sig', index_col=0)

In [10]:
df

,아이디,나이,성별,선호제품,주문상품명,옵션입력,앰플,건성,모름,복합성,지성,건조함,과잉피지,여드름,좁쌀,홍조&예민,기미&잡티
0,thsk486,4.0,0,진정앰플,[투비리본X아이다] 앰플&크림 2차 공구!,필수 앰플 쑥앰플 진정앰플 수분라이트 앰플 쑥앰플 진...,"[' 쑥앰플', ' 진정앰플', ' 쑥앰플', ' 진정앰플', ' 쑥앰플', ' 진...",1,0,0,0,1,0,0,1,1,0
1,thsk486,4.0,0,진정앰플,[투비리본X아이다] 앰플&크림 2차 공구!,필수 앰플 쑥앰플 진정앰플 수분라이트 앰플 쑥앰플 진...,"[' 쑥앰플', ' 진정앰플', ' 쑥앰플', ' 진정앰플', ' 쑥앰플', ' 진...",1,0,0,0,1,0,0,1,1,0
2,sjh3460,3.0,0,진정앰플,[투비리본X아이다] 앰플&크림 2차 공구!,필수 앰플 쑥앰플 진정앰플 수분라이트 앰플 쑥앰플 진...,"[' 쑥앰플', ' 진정앰플', ' 쑥앰플', ' 진정앰플', ' 쑥앰플', ' 진...",0,0,0,1,0,0,1,0,0,0
3,kaeng2da,3.0,0,수분앰플,[투비리본X아이다] 앰플&크림 2차 공구!,필수 앰플 쑥앰플 진정앰플 수분라이트 앰플 쑥앰플 진...,"[' 쑥앰플', ' 진정앰플', ' 쑥앰플', ' 진정앰플', ' 쑥앰플', ' 진...",0,0,1,0,0,0,0,1,0,0
4,gmlwlss625,2.0,0,진정앰플,[투비리본X아이다] 앰플&크림 2차 공구!,필수 앰플 쑥앰플 진정앰플 수분라이트 앰플 쑥앰플 진...,"[' 쑥앰플', ' 진정앰플', ' 쑥앰플', ' 진정앰플', ' 쑥앰플', ' 진...",0,0,1,0,0,1,1,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34651,nadaseonho,3.0,0,진정앰플,[시앙제이X아이다] 인생 버블토너 1차 공구!,필수 옵션 프로폴리스 진정토너 옵션 머그워트 클리어토너 옵션 ...,[' 진정앰플울트라'],0,0,1,0,1,0,0,0,1,0
34652,rarrarra12,3.0,0,진정앰플,BEST앰플3종(수분/미백/진정),필수 쿠폰할인 수분앰플 쿠폰할인 미백앰플 쿠폰할인 진정앰플,"[' 수분앰플', ' 미백앰플', ' 진정앰플']",0,0,0,1,0,1,1,1,0,0
34653,gy6686,1.0,0,진정앰플,주경민 이사님 개인결제창,본품 수분앰플오리지널 수분앰플라이트 진정앰플 모공 쑥 앰플 부스터앰플 미백앰플 진정...,"[' 수분앰플오리지널', ' 수분앰플라이트', ' 진정앰플', ' 부스터앰플', '...",1,0,0,0,1,0,0,0,1,0
34654,gy6686,1.0,0,진정앰플,주경민 이사님 개인결제창,본품 수분앰플오리지널 수분앰플라이트 진정앰플 모공 쑥 앰플 부스터앰플 미백앰플 진정...,"[' 수분앰플오리지널', ' 수분앰플라이트', ' 진정앰플', ' 부스터앰플', '...",1,0,0,0,1,0,0,0,1,0


In [11]:
# 앰플 더미변수 열 추가
df['쑥앰플'] = None
df['진정앰플'] = None
df['수분앰플'] = None
df['수분라이트'] = None
df['미백앰플'] = None

# 앰플 리스트
ampoul = ['쑥앰플', '진정앰플', '수분앰플', '수분라이트', '미백앰플']

# 수분앰플라이트 -> 수분라이트
df['앰플'] = df['앰플'].str.replace(pat='수분앰플라이트', repl='수분라이트', regex=False)

# 앰플 숫자형태로 인코딩
for i in range(len(df)):
    for j in ampoul:
        if j in df['앰플'][i]:
            df[j][i] = 1
        else:
            df[j][i] = 0

# 앰플 기준 빈칸 행 제거
for i in range(len(df)):

    if (df['진정앰플'][i] == 0 and df['수분앰플'][i] == 0 and df['미백앰플'][i] == 0 and df['수분라이트'][i] == 0 and df['쑥앰플'][i] == 0):
        df.drop(i, 0, inplace=True)
        
# 원래 앰플 열 제거
del df['앰플']

<ipython-input-11-647262f3d84c>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[j][i] = 1
<ipython-input-11-647262f3d84c>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[j][i] = 0


In [14]:
# 중간2 저장
df.to_csv("data/CRM_mid2.csv", encoding='utf-8-sig')

In [15]:
# 중간2 불러오기
df = pd.read_csv('data/CRM_mid2.csv', encoding='utf-8-sig',index_col=0)

In [16]:
df

,아이디,나이,성별,선호제품,주문상품명,옵션입력,건성,모름,복합성,지성,...,과잉피지,여드름,좁쌀,홍조&예민,기미&잡티,쑥앰플,진정앰플,수분앰플,수분라이트,미백앰플
0,thsk486,4.0,0,진정앰플,[투비리본X아이다] 앰플&크림 2차 공구!,필수 앰플 쑥앰플 진정앰플 수분라이트 앰플 쑥앰플 진...,1,0,0,0,...,0,0,1,1,0,1,1,0,0,0
1,thsk486,4.0,0,진정앰플,[투비리본X아이다] 앰플&크림 2차 공구!,필수 앰플 쑥앰플 진정앰플 수분라이트 앰플 쑥앰플 진...,1,0,0,0,...,0,0,1,1,0,1,1,0,0,0
2,sjh3460,3.0,0,진정앰플,[투비리본X아이다] 앰플&크림 2차 공구!,필수 앰플 쑥앰플 진정앰플 수분라이트 앰플 쑥앰플 진...,0,0,0,1,...,0,1,0,0,0,1,1,0,0,0
3,kaeng2da,3.0,0,수분앰플,[투비리본X아이다] 앰플&크림 2차 공구!,필수 앰플 쑥앰플 진정앰플 수분라이트 앰플 쑥앰플 진...,0,0,1,0,...,0,0,1,0,0,1,1,0,0,0
4,gmlwlss625,2.0,0,진정앰플,[투비리본X아이다] 앰플&크림 2차 공구!,필수 앰플 쑥앰플 진정앰플 수분라이트 앰플 쑥앰플 진...,0,0,1,0,...,1,1,1,1,0,1,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34651,nadaseonho,3.0,0,진정앰플,[시앙제이X아이다] 인생 버블토너 1차 공구!,필수 옵션 프로폴리스 진정토너 옵션 머그워트 클리어토너 옵션 ...,0,0,1,0,...,0,0,0,1,0,0,1,0,0,0
34652,rarrarra12,3.0,0,진정앰플,BEST앰플3종(수분/미백/진정),필수 쿠폰할인 수분앰플 쿠폰할인 미백앰플 쿠폰할인 진정앰플,0,0,0,1,...,1,1,1,0,0,0,1,1,0,1
34653,gy6686,1.0,0,진정앰플,주경민 이사님 개인결제창,본품 수분앰플오리지널 수분앰플라이트 진정앰플 모공 쑥 앰플 부스터앰플 미백앰플 진정...,1,0,0,0,...,0,0,0,1,0,0,1,1,1,1
34654,gy6686,1.0,0,진정앰플,주경민 이사님 개인결제창,본품 수분앰플오리지널 수분앰플라이트 진정앰플 모공 쑥 앰플 부스터앰플 미백앰플 진정...,1,0,0,0,...,0,0,0,1,0,0,1,1,1,1


In [17]:
# 아이디로 정렬
df = df.sort_values(by='아이디')

In [19]:
# 각 행별 구매 앰플 합산 저장 및 출력
id_ampoul = df[['아이디','쑥앰플', '진정앰플', '수분앰플', '수분라이트', '미백앰플']]
id_ampoul = id_ampoul.groupby(['아이디'], as_index=False).sum()
id_ampoul

,아이디,쑥앰플,진정앰플,수분앰플,수분라이트,미백앰플
0,10071809@n,12,26,37,15,25
1,10317295@n,0,5,3,3,0
2,10833064@n,0,10,0,6,1
3,109648536@n,0,7,7,3,2
4,109692597@n,1,10,6,3,4
...,...,...,...,...,...,...
3967,zzolmath,0,0,1,0,1
3968,zzz0812,0,3,0,3,0
3969,zzz9207,0,9,6,3,0
3970,zzzz092529,0,8,3,3,2


In [20]:
# idxmax() 사용하기 위해 문자가 포함 된 열 빼놓음
id = id_ampoul['아이디']
del id_ampoul['아이디']

id_ampoul

,쑥앰플,진정앰플,수분앰플,수분라이트,미백앰플
0,12,26,37,15,25
1,0,5,3,3,0
2,0,10,0,6,1
3,0,7,7,3,2
4,1,10,6,3,4
...,...,...,...,...,...
3967,0,0,1,0,1
3968,0,3,0,3,0
3969,0,9,6,3,0
3970,0,8,3,3,2


In [21]:
# 가장 많이 구매한 앰플 찾기
ampoul = id_ampoul.idxmax(axis=1)

# 아이디 중복 제거 (꼭 id_ampoul 처리 후)
df.drop_duplicates(['아이디'], keep = 'first', ignore_index=True, inplace=True)

# 각 행별 가장 많이 구매된 앰플 열 합치기
df_final = pd.concat([df, ampoul], axis=1, ignore_index=False)

In [23]:
# 기존 앰플 삭제 (레이블 값이 중복선택 되었기 때문)
del df_final['수분앰플'], df_final['쑥앰플'], df_final['진정앰플'], df_final['수분라이트'], df_final['미백앰플']

In [25]:
# 더미화 ('0'열 = ampoul)
encoding = pd.get_dummies(df_final[0])

# 앰플 더미변수 열 합치기
df_final = pd.concat([df_final, encoding], axis=1)

In [28]:
df_final

,나이,성별,건성,모름,복합성,지성,건조함,과잉피지,여드름,좁쌀,홍조&예민,기미&잡티,미백앰플,수분라이트,수분앰플,쑥앰플,진정앰플
0,3.0,0,1,0,0,0,1,0,0,1,0,0,0,0,1,0,0
1,3.0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,1
2,4.0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,1
3,1.0,0,0,0,1,0,1,0,1,1,1,1,0,0,0,0,1
4,1.0,0,0,0,0,1,0,1,0,1,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3967,3.0,1,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0
3968,8.0,0,0,0,1,0,0,1,1,0,0,1,0,0,0,0,1
3969,3.0,0,0,0,0,1,0,0,1,1,1,0,0,0,0,0,1
3970,2.0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,0,1


In [27]:
# 필요없는 열 삭제
del df_final['아이디'], df_final['선호제품'], df_final['주문상품명'], df_final['옵션입력'], df_final[0]

In [168]:
# 모름 열 삭제
#idx_nm_1 = df_final[df_final['모름']==1].index
#df_final = df_final.drop(idx_nm_1)
#del df_final['모름']

In [29]:
# 최종 저장
df_final.to_csv('data/CRM_final.csv', encoding='utf-8-sig')

### 다중 분류 알고리즘(softmax)

In [31]:
# y_data 저장
y_data = df_final.iloc[:, 12:17]
y_data

,미백앰플,수분라이트,수분앰플,쑥앰플,진정앰플
0,0,0,1,0,0
1,0,0,0,0,1
2,0,0,0,0,1
3,0,0,0,0,1
4,0,0,0,0,1
...,...,...,...,...,...
3967,0,0,1,0,0
3968,0,0,0,0,1
3969,0,0,0,0,1
3970,0,0,0,0,1


In [35]:
# x_data 저장
x_data = df_final.iloc[:, :12]
x_data

,나이,성별,건성,모름,복합성,지성,건조함,과잉피지,여드름,좁쌀,홍조&예민,기미&잡티
0,3.0,0,1,0,0,0,1,0,0,1,0,0
1,3.0,0,0,0,1,0,0,1,0,1,0,0
2,4.0,0,0,0,1,0,0,0,0,1,0,1
3,1.0,0,0,0,1,0,1,0,1,1,1,1
4,1.0,0,0,0,0,1,0,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
3967,3.0,1,0,0,1,0,0,0,1,0,0,0
3968,8.0,0,0,0,1,0,0,1,1,0,0,1
3969,3.0,0,0,0,0,1,0,0,1,1,1,0
3970,2.0,0,1,0,0,0,0,0,0,1,1,0


In [55]:
# 데이터 셋 나누기
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.3, random_state=123, stratify=y_data)

In [63]:
# 배열로 바꾸기, 데이터 타입 float으로 바꾸기
X = np.asarray(X).astype(np.float32)
y = np.asarray(y).astype(np.float32)
x_test = np.asarray(x_test).astype(np.float32)
y_test = np.asarray(y_test).astype(np.float32)

In [89]:
# 모델링
model=Sequential()
model.add(Dense(64, input_dim=12, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(5, activation='softmax'))

model.compile(optimizer=Adam(lr=0.01) ,loss='categorical_crossentropy',metrics=['accuracy'])
# SGD. RMSprop, Adam
# 옵티마이저는 경사하강법의 일종인 Adam을 사용한다
# 손실 함수(Loss function)는 크로스 엔트로피 함수를 사용한다.
history = model.fit(X,y, batch_size=1, epochs=200, validation_data=(x_test, y_test))
# 주어진 X와 y데이터에 대해서 오차를 최소화하는 작업을 200번 시도한다.

Epoch 1/200
2780/2780 [==============================] - 3s 996us/step - loss: 0.9371 - accuracy: 0.7162 - val_loss: 0.8132 - val_accuracy: 0.7349
Epoch 2/200
2780/2780 [==============================] - 2s 878us/step - loss: 0.8369 - accuracy: 0.7369 - val_loss: 0.8277 - val_accuracy: 0.7349
Epoch 3/200
2780/2780 [==============================] - 3s 925us/step - loss: 0.7966 - accuracy: 0.7424 - val_loss: 0.8470 - val_accuracy: 0.7349
Epoch 4/200
2780/2780 [==============================] - 3s 924us/step - loss: 0.8148 - accuracy: 0.7341 - val_loss: 0.8122 - val_accuracy: 0.7349
Epoch 5/200
2780/2780 [==============================] - 3s 913us/step - loss: 0.8012 - accuracy: 0.7437 - val_loss: 0.8092 - val_accuracy: 0.7349
Epoch 6/200
2780/2780 [==============================] - 3s 922us/step - loss: 0.8081 - accuracy: 0.7298 - val_loss: 0.8525 - val_accuracy: 0.7341
Epoch 7/200
2780/2780 [==============================] - 3s 924us/step - loss: 0.7845 - accuracy: 0.7495 - val_loss: 0

2780/2780 [==============================] - 3s 948us/step - loss: 0.7985 - accuracy: 0.7492 - val_loss: 1.8125 - val_accuracy: 0.7341
Epoch 57/200
2780/2780 [==============================] - 3s 947us/step - loss: 0.8236 - accuracy: 0.7359 - val_loss: 1.8685 - val_accuracy: 0.7341
Epoch 58/200
2780/2780 [==============================] - 3s 947us/step - loss: 0.8317 - accuracy: 0.7346 - val_loss: 1.9791 - val_accuracy: 0.7341
Epoch 59/200
2780/2780 [==============================] - 3s 971us/step - loss: 0.8270 - accuracy: 0.7409 - val_loss: 1.8473 - val_accuracy: 0.7341
Epoch 60/200
2780/2780 [==============================] - 3s 916us/step - loss: 0.8108 - accuracy: 0.7374 - val_loss: 2.0676 - val_accuracy: 0.7341
Epoch 61/200
2780/2780 [==============================] - 3s 925us/step - loss: 0.8372 - accuracy: 0.7231 - val_loss: 1.7396 - val_accuracy: 0.7341
Epoch 62/200
2780/2780 [==============================] - 3s 952us/step - loss: 0.8257 - accuracy: 0.7308 - val_loss: 2.1673 

2780/2780 [==============================] - 3s 966us/step - loss: 0.7761 - accuracy: 0.7555 - val_loss: 3.3097 - val_accuracy: 0.7349
Epoch 112/200
2780/2780 [==============================] - 3s 945us/step - loss: 0.7945 - accuracy: 0.7385 - val_loss: 3.9927 - val_accuracy: 0.7349
Epoch 113/200
2780/2780 [==============================] - 3s 961us/step - loss: 0.8063 - accuracy: 0.7449 - val_loss: 4.1469 - val_accuracy: 0.7349
Epoch 114/200
2780/2780 [==============================] - 3s 948us/step - loss: 0.8219 - accuracy: 0.7316 - val_loss: 4.9828 - val_accuracy: 0.7349
Epoch 115/200
2780/2780 [==============================] - 3s 937us/step - loss: 0.7982 - accuracy: 0.7387 - val_loss: 5.3477 - val_accuracy: 0.7349
Epoch 116/200
2780/2780 [==============================] - 3s 941us/step - loss: 0.8467 - accuracy: 0.7199 - val_loss: 5.8344 - val_accuracy: 0.7349
Epoch 117/200
2780/2780 [==============================] - 3s 955us/step - loss: 0.8141 - accuracy: 0.7331 - val_loss: 6

2780/2780 [==============================] - 3s 1ms/step - loss: 0.7913 - accuracy: 0.7384 - val_loss: 7.6727 - val_accuracy: 0.7341
Epoch 167/200
2780/2780 [==============================] - 3s 942us/step - loss: 0.8839 - accuracy: 0.7214 - val_loss: 6.9138 - val_accuracy: 0.7349
Epoch 168/200
2780/2780 [==============================] - 3s 921us/step - loss: 0.8080 - accuracy: 0.7389 - val_loss: 7.3487 - val_accuracy: 0.7349
Epoch 169/200
2780/2780 [==============================] - 3s 951us/step - loss: 0.7968 - accuracy: 0.7374 - val_loss: 7.5363 - val_accuracy: 0.7349
Epoch 170/200
2780/2780 [==============================] - 3s 953us/step - loss: 0.7814 - accuracy: 0.7436 - val_loss: 7.5671 - val_accuracy: 0.7349
Epoch 171/200
2780/2780 [==============================] - 3s 978us/step - loss: 0.7890 - accuracy: 0.7458 - val_loss: 7.6185 - val_accuracy: 0.7349
Epoch 172/200
2780/2780 [==============================] - 3s 960us/step - loss: 0.8211 - accuracy: 0.7228 - val_loss: 7.6

In [58]:
print(model.predict(X))
y

[[0.06396758 0.00409223 0.1844258  0.02453044 0.72298396]
 [0.06396758 0.00409223 0.1844258  0.02453044 0.72298396]
 [0.06396758 0.00409223 0.1844258  0.02453044 0.72298396]
 ...
 [0.06396758 0.00409223 0.1844258  0.02453044 0.72298396]
 [0.06396758 0.00409223 0.1844258  0.02453044 0.72298396]
 [0.06396758 0.00409223 0.1844258  0.02453044 0.72298396]]


array([[0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       [0., 0., 1., 0., 0.],
       ...,
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.]], dtype=float32)